## <font color='red'> INSTRUCTIONS </font>

<b> 
1. Write your code only in cells below the "WRITE CODE BELOW" title. Do not modify the code below the "DO NOT MODIFY" title. <br>
2. The expected data types of the output answers for each question are given in the last cell through assertion statements. Your answers must match these expected output data types. Hint: Many of the answers need to be a Python dictionary. Consider methods like to_dict() to convert a Pandas Series to a dictionary. <br>
3. The answers are then written to a JSON file named my_results_PA1.json. You can compare this with the provided expected output file "expected_results_PA1.json". <br>
4. After you complete writing your code, click "Kernel -> Restart Kernel and Run All Cells" on the top toolbar. There should NOT be any syntax/runtime errors, otherwise points will be deducted. <br>
5. For submitting your solution, first download your notebook by clicking "File -> Download". Rename the file as &ltTEAM_ID&gt.ipynb" and upload to Canvas.</b>


## <font color='red'> DO NOT MODIFY </font>

In [2]:
import time
import json
import dask
import dask.dataframe as dd
import pandas as pd
import ast
import re
from dask.distributed import Client
import ctypes
import numpy as np

def trim_memory() -> int:
    """
    helps to fix any memory leaks.
    """
    libc = ctypes.CDLL("libc.so.6")
    return libc.malloc_trim(0)

client = Client("127.0.0.1:8786")
client.run(trim_memory)
client = client.restart()
print(client)

None


In [2]:
start = time.time()

## <font color='blue'> WRITE CODE BELOW </font>

In [90]:
### read in the 'user_reviews.csv' and 'products.csv' files, perform your calculations and place the answers in variables ans1 - ans7.


# substitute 'None' with the outputs from your calculations. 
# The expected output types can be seen in the assertion statements below
reviews = dd.read_csv('user_reviews.csv', dtype={'asin': 'object'})
products = dd.read_csv('products.csv', dtype={'asin': 'object'})


rev = reviews.partitions[0]
prod = products.partitions[0]

#Q1 + 2
ans1 = rev.isna().sum(axis=0) / len(rev) * 100
ans2 = prod.isna().sum(axis=0) / len(prod) * 100


#Q3
rev_small = rev[['asin', 'overall']]
prod_small = prod[['asin', 'price']]
merged = dd.merge(rev_small, prod_small, on='asin', how='inner')
merged = merged.dropna(subset=['overall', 'price'])
corr = merged[['overall', 'price']].corr()
ans3 = corr.loc['overall', 'price']

#Q4
price = prod['price'].dropna()
ans4 = {
    'mean': price.mean().compute(),
    'std': price.std().compute(),
    'min': price.min().compute(),
    'max': price.max().compute(),
    'median': price.quantile(0.5).compute()
}

#Q5
cat = prod['categories'].dropna()
def get_supercat(cat_str):
    try:
        cat_list = eval(cat_str)
        if isinstance(cat_list, list) and cat_list and isinstance(cat_list[0], list) and cat_list[0] and cat_list[0][0]:
            return cat_list[0][0]
        else:
            return None
    except:
        return None

valid_cats = cat.map(get_supercat, meta=('supercat', 'object'))
cat_count = valid_cats.value_counts().compute()
ans5 = cat_count.to_dict()

#Q6:
rev_asins = set(rev['asin'].dropna().unique().compute())
prod_asins = set(prod['asin'].dropna().unique().compute())
ans6 = int(len(rev_asins - prod_asins) > 0)

#Q7
related = prod['related'].dropna()

def extract_related_ids(related_str):
    try:
        d = ast.literal_eval(related_str)
        ids = []
        for k in ['also_bought', 'also_viewed', 'bought_together', 'buy_after_viewing']:
            if k in d and isinstance(d[k], list):
                ids.extend(d[k])
        return ids
    except:
        return []

related_ids = related.map(extract_related_ids, meta=('related_ids', 'object')).explode()
related_ids = related_ids.dropna()
related_ids_set = set(related_ids)
ans7 = int(len(related_ids_set - prod_asins_set) > 0)
ans7b

1

In [3]:
start = time.time()
reviews = dd.read_csv('user_reviews.csv', dtype={'asin': 'object'})
products = dd.read_csv('products.csv', dtype={'asin': 'object'})

reviews = reviews.persist()
products = products.persist()
end = time.time()
print(f"execution time = {end-start}s")


execution time = 0.15819239616394043s


In [5]:
from dask import compute

In [6]:
# ──────────────────────────────
# Q1, Q2: Percentage of missing values
# ──────────────────────────────
start = time.time()
q1 = reviews.isna().sum() / len(reviews) * 100
q2 = products.isna().sum() / len(products) * 100
ans1, ans2 = compute(q1, q2)
end = time.time()
print(f"execution time = {end-start}s")

execution time = 50.164355516433716s


In [8]:
ans2

asin            0.000000
salesRank      26.015663
imUrl           1.795239
categories      0.798038
title          15.192683
description    41.718038
price          35.703404
related        29.039236
brand          81.685090
dtype: float64

In [9]:
start = time.time()
# ──────────────────────────────
# Q3: Pearson correlation between price and overall rating
# ──────────────────────────────
merged = dd.merge(
    reviews[['asin', 'overall']],
    products[['asin', 'price']],
    on='asin',
    how='inner'
).dropna(subset=['overall', 'price'])

ans3 = merged[['overall', 'price']].corr().compute().loc['overall', 'price']
end = time.time()
print(f"execution time = {end-start}s")

execution time = 77.63178420066833s


In [11]:
start = time.time()
# ──────────────────────────────
# Q4: Summary stats for price
# ──────────────────────────────
price = products['price'].dropna()
q4_mean = price.mean()
q4_std = price.std()
q4_min = price.min()
q4_max = price.max()
q4_median = price.quantile(0.5)

q4_mean, q4_std, q4_min, q4_max, q4_median = compute(
    q4_mean, q4_std, q4_min, q4_max, q4_median
)

ans4 = {
    'mean': q4_mean,
    'std': q4_std,
    'min': q4_min,
    'max': q4_max,
    'median': q4_median
}
end = time.time()
print(f"execution time = {end-start}s")

execution time = 1.5656347274780273s


In [12]:
start = time.time()
# ──────────────────────────────
# Q5: Count of products by super-category
# ──────────────────────────────
def get_supercat(cat_str):
    try:
        cat_list = ast.literal_eval(cat_str)
        if isinstance(cat_list, list) and cat_list and isinstance(cat_list[0], list) and cat_list[0]:
            return cat_list[0][0]
    except:
        return None
    return None

valid_cats = products['categories'].dropna().map(get_supercat, meta=('supercat', 'object'))
ans5 = valid_cats.value_counts().compute().to_dict()
end = time.time()
print(f"execution time = {end-start}s")

execution time = 20.106683254241943s


In [14]:
start = time.time()
# ──────────────────────────────
# Q6: Dangling references from reviews → products
# ──────────────────────────────
rev_asins, prod_asins = compute(
    reviews['asin'].dropna().unique(),
    products['asin'].dropna().unique()
)
ans6 = int(len(set(rev_asins) - set(prod_asins)) > 0)
end = time.time()
print(f"execution time = {end-start}s")

execution time = 93.52458477020264s


In [ ]:
start = time.time()
# ──────────────────────────────
# Q7: Dangling references from related → products
# ──────────────────────────────
def extract_related_ids(related_str):
    try:
        d = ast.literal_eval(related_str)
        ids = []
        for k in ['also_bought', 'also_viewed', 'bought_together', 'buy_after_viewing']:
            if k in d and isinstance(d[k], list):
                ids.extend(d[k])
        return ids
    except:
        return []

prod_asins_set = set(products['asin'].dropna().unique().compute())

def is_dangling(asin):
    return asin not in prod_asins_set

# Apply lazily
dangling_flags = products['related'].dropna().map(
    extract_related_ids, meta=('related_ids', 'object')
).explode().dropna().map(is_dangling, meta=('dangling', 'bool'))

# If any True → dangling reference exists
ans7 = int(dangling_flags.any().compute())
end = time.time()
print(f"execution time = {end-start}s")

/home/ubuntu/dask_env/lib/python3.10/site-packages/distributed/client.py:3383: UserWarning: Sending large graph of size 117.04 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


In [3]:
import dask.dataframe as dd
import ast
from dask import compute

# Load data
reviews = dd.read_csv('user_reviews.csv', dtype={'asin': 'object'})
products = dd.read_csv('products.csv', dtype={'asin': 'object'})

reviews = reviews.persist()
products = products.persist()
# ──────────────────────────────
# Q1, Q2: Percentage of missing values
# ──────────────────────────────
q1 = reviews.isna().sum() / len(reviews) * 100
q2 = products.isna().sum() / len(products) * 100
ans1, ans2 = compute(q1, q2)

# ──────────────────────────────
# Q3: Pearson correlation between price and overall rating
# ──────────────────────────────
merged = dd.merge(
    reviews[['asin', 'overall']],
    products[['asin', 'price']],
    on='asin',
    how='inner'
).dropna(subset=['overall', 'price'])

ans3 = merged[['overall', 'price']].corr().compute().loc['overall', 'price']

# ──────────────────────────────
# Q4: Summary stats for price
# ──────────────────────────────
price = products['price'].dropna()
price_stats, median_price = compute(
    price.agg(['mean', 'std', 'min', 'max']),
    price.quantile(0.5)
)
ans4 = {
    'mean': price_stats['mean'],
    'std': price_stats['std'],
    'min': price_stats['min'],
    'max': price_stats['max'],
    'median': median_price
}

# ──────────────────────────────
# Q5: Count of products by super-category
# ──────────────────────────────
def get_supercat(cat_str):
    try:
        cat_list = ast.literal_eval(cat_str)
        if isinstance(cat_list, list) and cat_list and isinstance(cat_list[0], list) and cat_list[0]:
            return cat_list[0][0]
    except:
        return None
    return None

valid_cats = products['categories'].dropna().map(get_supercat, meta=('supercat', 'object'))
ans5 = valid_cats.value_counts().compute().to_dict()

# ──────────────────────────────
# Q6: Dangling references from reviews → products
# ──────────────────────────────
rev_asins, prod_asins = compute(
    reviews['asin'].dropna().unique(),
    products['asin'].dropna().unique()
)
ans6 = int(len(set(rev_asins) - set(prod_asins)) > 0)

# ──────────────────────────────
# Q7: Dangling references from related → products
# ──────────────────────────────
def extract_related_ids(related_str):
    try:
        d = ast.literal_eval(related_str)
        ids = []
        for k in ['also_bought', 'also_viewed', 'bought_together', 'buy_after_viewing']:
            if k in d and isinstance(d[k], list):
                ids.extend(d[k])
        return ids
    except:
        return []

related_ids = products['related'].dropna().map(
    extract_related_ids, meta=('related_ids', 'object')
).explode().dropna()

related_ids_set = set(related_ids.compute())
ans7 = int(len(related_ids_set - set(prod_asins)) > 0)

2025-05-13 00:51:41,288 - distributed.client - WARNING - Couldn't gather 418 keys, rescheduling (('read-e7c1646f42a971c9f96afbc29294f0af', 50), ('read-e7c1646f42a971c9f96afbc29294f0af', 59), ('read-e7c1646f42a971c9f96afbc29294f0af', 68), ('read-e7c1646f42a971c9f96afbc29294f0af', 306), ('read-e7c1646f42a971c9f96afbc29294f0af', 315), ('read-9e8a220db3db5f2989593b5b681f8b5e', 1), ('read-9e8a220db3db5f2989593b5b681f8b5e', 10), ('read-9e8a220db3db5f2989593b5b681f8b5e', 19), ('read-9e8a220db3db5f2989593b5b681f8b5e', 28), ('read-9e8a220db3db5f2989593b5b681f8b5e', 37), ('read-e7c1646f42a971c9f96afbc29294f0af', 171), ('read-e7c1646f42a971c9f96afbc29294f0af', 180), ('read-e7c1646f42a971c9f96afbc29294f0af', 189), ('read-e7c1646f42a971c9f96afbc29294f0af', 198), ('read-e7c1646f42a971c9f96afbc29294f0af', 207), ('read-9e8a220db3db5f2989593b5b681f8b5e', 131), ('read-9e8a220db3db5f2989593b5b681f8b5e', 140), ('read-e7c1646f42a971c9f96afbc29294f0af', 36), ('read-9e8a220db3db5f2989593b5b681f8b5e', 149), (

KeyboardInterrupt: 

In [94]:
import dask.dataframe as dd
import ast

# Read data
reviews = dd.read_csv('user_reviews.csv', dtype={'asin': 'object'})
products = dd.read_csv('products.csv', dtype={'asin': 'object'})

# Q1, Q2
q1 = reviews.isna().sum() / len(reviews) * 100
q2 = products.isna().sum() / len(products) * 100

# Q3
merged = dd.merge(
    reviews[['asin', 'overall']],
    products[['asin', 'price']],
    on='asin',
    how='inner'
).dropna(subset=['overall', 'price'])
q3 = merged[['overall', 'price']].corr()

# Q4
price = products['price'].dropna()
q4_mean = price.mean()
q4_std = price.std()
q4_min = price.min()
q4_max = price.max()
q4_median = price.quantile(0.5)

# Q5
def get_supercat(cat_str):
    try:
        cat_list = ast.literal_eval(cat_str)
        if isinstance(cat_list, list) and cat_list and isinstance(cat_list[0], list) and cat_list[0]:
            return cat_list[0][0]
    except:
        return None
    return None

valid_cats = products['categories'].dropna().map(get_supercat, meta=('supercat', 'object'))
q5 = valid_cats.value_counts()

# Q6
rev_asins = reviews['asin'].dropna().unique()
prod_asins = products['asin'].dropna().unique()

# Q7
def extract_related_ids(related_str):
    try:
        d = ast.literal_eval(related_str)
        ids = []
        for k in ['also_bought', 'also_viewed', 'bought_together', 'buy_after_viewing']:
            if k in d and isinstance(d[k], list):
                ids.extend(d[k])
        return ids
    except:
        return []

related_ids = products['related'].dropna().map(extract_related_ids, meta=('related_ids', 'object')).explode().dropna()

# Combine all expressions
unified = (
    q1, q2, q3,
    q4_mean, q4_std, q4_min, q4_max, q4_median,
    q5,
    rev_asins, prod_asins,
    related_ids
)

# Visualize the full DAG
dask.visualize(unified, filename='full_pa1_dag', format='png', rankdir='TB')

AttributeError: module 'dask.dataframe' has no attribute 'visualize'

In [ ]:
which dot

In [97]:
# Visualize the full DAG
dask.visualize(unified, filename='full_pa1_dag', format='png', rankdir='TB')

ExecutableNotFound: failed to execute PosixPath('dot'), make sure the Graphviz executables are on your systems' PATH

In [92]:
ans1#, ans2, ans3, ans4, ans5, ans6, ans7

Dask Series Structure:
npartitions=1
asin              float64
unixReviewTime        ...
Dask Name: mul, 7 expressions
Expr=(IsNa(frame=Partitions(frame=ArrowStringConversion(frame=FromMapProjectable(df2bcf2)), partitions=[0]))).sum() / 111390 * 100

In [22]:
#Question1
rev.isna().sum(axis=0) / len(rev)

reviewerID        0.000000
asin              0.000000
reviewerName      0.033450
helpful           0.000000
reviewText        0.000180
overall           0.000000
summary           0.000117
unixReviewTime    0.000000
reviewTime        0.000000
dtype: float64

In [21]:
#Q2:
prod.isna().sum(axis=0) / len(prod)

asin           0.000000
salesRank      0.003960
imUrl          0.016298
categories     0.000271
title          0.000625
description    0.281278
price          0.234530
related        0.185240
brand          0.999812
dtype: float64

In [24]:
#Q3:
rev_small = rev[['asin', 'overall']]
prod_small = prod[['asin', 'price']]

merged = pd.merge(rev_small, prod_small, on='asin', how='inner')

merged = merged.dropna(subset=['overall', 'price'])
corr = merged[['overall', 'price']].corr()
corr.loc['overall', 'price']

np.float64(-0.023792516709690775)

In [31]:
#Q4:
price = prod['price'].dropna()
price.describe()#[['mean', 'std', 'median', 'min', 'max']]
mean = price.mean()
std = price.std()
pmin = price.min()
pmax = price.max()
median = price.median()
{
    'mean': mean,
    'std': std,
    'min': pmin,
    'max': pmax,
    'median': median,
}

{'mean': np.float64(19.031475985624045),
 'std': np.float64(30.775630420960038),
 'min': np.float64(0.0),
 'max': np.float64(888.0),
 'median': np.float64(10.67)}

In [69]:
#Q5:
import ast
cat = prod['categories'].dropna().compute()


def get_supercat(cat_str):
    try:
        cat_list = eval(cat_str)
        if isinstance(cat_list, list) and cat_list and isinstance(cat_list[0], list) and cat_list[0] and cat_list[0][0]:
            return cat_list[0][0]
        else:
            return None
    except:
        return None

valid_cats = cat.map(get_supercat)
valid_cats

cat_count = valid_cats.value_counts().sort_values(ascending=False)
cat_count.to_dict

<bound method Series.to_dict of categories
Books                        47894
Movies & TV                     46
CDs & Vinyl                      8
Sports & Outdoors                6
Musical Instruments              4
Toys & Games                     2
Clothing, Shoes & Jewelry        2
Tools & Home Improvement         1
Software                         1
Name: count, dtype: int64>

In [60]:
#Q6:
rev_asins = rev['asin'].dropna()
prod_asins = prod['asin'].dropna()
rev_asins_set = set(rev_asins)
prod_asins_set = set(prod_asins)
dangling_refs = rev_asins_set - prod_asins_set
int(len(dangling_refs) >0)

1

In [86]:
#Q7:
related = prod['related'].dropna()

def extract_related_ids(related_str):
    try:
        d = ast.literal_eval(related_str)
        ids = []
        for k in ['also_bought', 'also_viewed', 'bought_together', 'buy_after_viewing']:
            if k in d and isinstance(d[k], list):
                ids.extend(d[k])
        return ids
    except:
        return []

related_ids = related.map(extract_related_ids, meta=('related_ids', 'object')).explode()
related_ids = related_ids.dropna()
related_ids_set = set(related_ids.compute())
ans7 = int(len(related_ids_set - prod_asins_set) > 0)
ans7

1

## <font color='red'> DO NOT MODIFY </font>

In [ ]:
end = time.time()

In [ ]:
print(f"execution time = {end-start}s")

In [ ]:
# DO NOT MODIFY
assert type(ans1) == dict, f"answer to question 1 must be a dictionary like {{'reviewerID':0.2, ..}}, got type = {type(ans1)}"
assert type(ans2) == dict, f"answer to question 2 must be a dictionary like {{'asin':0.2, ..}}, got type = {type(ans2)}"
assert type(ans3) == float, f"answer to question 3 must be a float like 0.8, got type = {type(ans3)}"
assert type(ans4) == dict, f"answer to question 4 must be a dictionary like {{'mean':0.4,'max':0.6,'median':0.6...}}, got type = {type(ans4)}"
assert type(ans5) == dict, f"answer to question 5 must be a dictionary, got type = {type(ans5)}"         
assert ans6 == 0 or ans6==1, f"answer to question 6 must be 0 or 1, got value = {ans6}" 
assert ans7 == 0 or ans7==1, f"answer to question 7 must be 0 or 1, got value = {ans7}" 

ans_dict = {
    "q1": ans1,
    "q2": ans2,
    "q3": ans3,
    "q4": ans4,
    "q5": ans5,
    "q6": ans6,
    "q7": ans7,
    "runtime": end-start
}
with open('my_results_PA1.json', 'w') as outfile: json.dump(ans_dict, outfile)         